In [4]:
import torch
import numpy as np
import math

In [6]:
class Embedding(torch.nn.Module):
    def __init__(self, vocab_size, embed_size, dropout=0.1):
        super(Embedding,self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.dropout = torch.nn.Dropout(p=dropout)
        self.lut = torch.nn.Embedding(vocab_size, embed_size)

    def forward(self,x):
        x = self.lut(x)
        x = self.dropout(x)
        return x * math.sqrt(self.embed_size)

In [8]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)
        # 先创建一个全零矩阵
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        # 用sin cos来填充原来的pos矩阵
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        # 这样我们就得到了位置编码矩阵pe, pe现在还只是一个二维矩阵，要想和embedding的输出（一个三维张量）相加，
        # 就必须拓展一个维度，所以这里使用unsqueeze拓展维度.
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)


    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_size, dropout=0.1):
        super(Encoder,self).__init__()